In [4]:
# Add root to path using sys
import sys
sys.path.append("../")

import sqlite3
import pandas as pd
from src.cmesrc.config import CMESRC_DB

In [5]:
con = sqlite3.connect(CMESRC_DB)
con.execute("PRAGMA foreign_keys = ON")
cursor = con.cursor()

In [6]:
cursor.execute("SELECT COUNT(*) FROM harps_pixel_bbox")
print(cursor.fetchone()[0])

2628572


In [16]:
df = pd.read_sql_query("SELECT harpnum, (x_max - x_min) AS width, (y_max - y_min) AS height FROM harps_pixel_bbox WHERE harpnum = 1", con)
df.width.max()

58

In [21]:
cursor.execute("DROP TABLE IF EXISTS harps_pixel_bbox_sizes")
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS harps_pixel_bbox_sizes AS
    SELECT harpnum, MAX(x_max - x_min) AS width, MAX(y_max - y_min) AS height
    FROM harps_pixel_bbox
    GROUP BY harpnum
    """
)

df = pd.read_sql_query("SELECT * FROM harps_pixel_bbox_sizes LIMIT 100", con)
df

,harpnum,width,height
0,1,58,30
1,2,41,31
2,5,31,10
3,6,25,14
4,8,54,19
...,...,...,...
95,166,13,8
96,168,43,32
97,171,9,5
98,173,11,5


In [22]:
cursor.execute("DROP TABLE IF EXISTS longitude_valid_cutouts")
cursor.execute("DROP TABLE IF EXISTS cutouts_for_download")

cursor.execute(
    """
    CREATE TEMPORARY TABLE IF NOT EXISTS longitude_valid_cutouts AS
    SELECT hpb.* FROM harps_pixel_bbox hpb
    INNER JOIN harps_bbox hb ON hpb.timestamp = hb.timestamp AND hpb.harpnum = hb.harpnum
    WHERE (hb.londtmin + hb.londtmax) / 2 BETWEEN -50 AND 50
    """)

cursor.execute("CREATE INDEX IF NOT EXISTS idx_lvc_harpnum ON longitude_valid_cutouts (harpnum)")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_lvc_timestamp ON longitude_valid_cutouts (timestamp)")

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS cutouts_for_download AS
    WITH cutout_durations AS (
        SELECT harpnum,
            (MAX(strftime('%s', timestamp)) - MIN(strftime('%s', timestamp))) / 60 / 60 / 24 as day_range
            FROM longitude_valid_cutouts lvc
            GROUP BY harpnum
    )

    SELECT lvc.harpnum, lvc.timestamp
    FROM longitude_valid_cutouts lvc
    INNER JOIN cutout_durations cd ON lvc.harpnum = cd.harpnum
    WHERE cd.day_range > 4
    """
)

cursor.execute("SELECT COUNT(*) FROM cutouts_for_download")
print(cursor.fetchone()[0])

1552831


In [23]:
df = pd.read_sql_query(
    """
    SELECT hpb.*, i.idx FROM harps_pixel_bbox hpb
    INNER JOIN cutouts_for_download cfd ON hpb.timestamp = cfd.timestamp AND hpb.harpnum = cfd.harpnum
    INNER JOIN images i ON hpb.timestamp = i.timestamp
    LIMIT 10""", con)
            
df

,harpnum,timestamp,x_min,x_max,y_min,y_max,x_cen,y_cen,idx
0,1,2010-05-03 04:48:00,92,137,333,361,114,347,234
1,1,2010-05-03 05:00:00,92,137,333,361,114,347,235
2,1,2010-05-03 05:12:00,92,137,333,361,114,347,236
3,1,2010-05-03 05:24:00,93,137,333,361,115,347,237
4,1,2010-05-03 05:36:00,93,138,333,361,115,347,238
5,1,2010-05-03 05:48:00,93,138,333,361,115,347,239
6,1,2010-05-03 06:24:00,93,139,333,361,116,347,240
7,1,2010-05-03 06:36:00,94,139,333,361,116,347,241
8,1,2010-05-03 06:48:00,94,139,333,361,116,347,242
9,1,2010-05-03 07:00:00,94,140,333,361,117,347,243


In [10]:
con.commit()
con.close()